In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Trained on 56 features
These features were shown to affect the Target, or if somone is likely to default, via Lasso Regression.



In [55]:
app_data_1 = pd.read_csv("X_train(num2).csv")
app_data_1.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BUILD_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,YEARS_BUILD_MEDI,FONDKAPREMONT_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,DEF_30_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_16,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,0.0,0.5,0.0,1.0,0.245232,0.489166,0.537494,0.468924,1.000000,1.000000,1.00,0.6,0.2,0.263040,0.888839,0.852140,0.705433,0.509220,1.0,0.0,1.0,0.0,0.444444,0.000000,0.5,0.5,1.000000,0.0,0.0,0.0,0.0,0.087719,0.098970,0.377820,0.203299,0.708496,0.096262,0.115433,0.699818,0.75,0.857143,0.5,0.058824,0.083333,0.735788,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212746
1,100003,0,0.0,0.0,0.0,0.0,0.279376,0.746352,0.610092,0.722714,0.142857,0.571429,0.25,0.2,0.2,0.046547,0.477114,0.951929,0.959566,0.509220,1.0,0.0,1.0,0.0,0.166667,0.052632,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.684211,0.388769,0.783013,0.669848,0.851199,0.048934,0.369271,0.846955,0.75,0.000000,0.5,0.000000,0.000000,0.807083,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,100004,0,1.0,0.5,1.0,1.0,0.114839,0.244144,0.281783,0.261437,1.000000,1.000000,1.00,0.6,0.2,0.139034,0.348534,0.827335,0.648326,0.728879,1.0,1.0,1.0,0.0,0.444444,0.000000,0.5,0.5,0.166667,0.0,0.0,0.0,0.0,0.192982,0.598606,0.715444,0.856262,0.819505,0.186374,0.222434,0.814345,0.00,0.142857,0.0,0.000000,0.000000,0.810112,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,100006,0,0.0,0.0,0.0,1.0,0.197108,0.430800,0.573739,0.432648,1.000000,1.000000,1.00,0.0,0.2,0.110416,0.350846,0.601451,0.661387,0.509220,1.0,0.0,0.0,0.0,0.444444,0.052632,0.5,0.5,1.000000,0.0,0.0,0.0,0.0,0.087719,0.598606,0.810901,0.669848,0.819505,0.186374,0.222434,0.814345,0.00,0.142857,0.0,0.000000,0.000000,0.856244,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212746
4,100007,0,0.0,0.5,0.0,1.0,0.184602,0.540824,0.513463,0.551329,1.000000,1.000000,1.00,0.6,0.2,0.401234,0.298591,0.825268,0.519522,0.509220,1.0,0.0,0.0,0.0,0.166667,0.000000,0.5,0.5,0.666667,0.0,0.0,0.0,1.0,0.649123,0.598606,0.452680,0.669848,0.819505,0.186374,0.222434,0.814345,0.00,0.142857,0.0,0.000000,0.000000,0.742311,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,0.0,0.5,0.0,0.0,0.215404,0.385219,0.559073,0.372361,1.000000,1.000000,1.00,0.4,1.0,0.455490,0.896392,0.657263,0.724607,0.509220,1.0,0.0,0.0,0.0,0.833333,0.000000,0.0,0.0,0.666667,0.0,0.0,0.0,0.0,0.754386,0.184040,0.841201,0.669848,0.776104,0.048934,0.544288,0.873656,0.75,0.857143,0.5,0.000000,0.000000,0.936393,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212746
307507,456252,0,0.0,0.0,0.0,1.0,0.122499,0.397813,0.395216,0.372361,1.000000,0.428571,1.00,1.0,0.2,0.352355,0.251071,0.822147,0.431708,0.509220,0.0,0.0,1.0,0.0,0.666667,0.000000,0.5,0.5,0.166667,0.0,0.0,0.0,0.0,1.000000,0.598606,0.177612,0.669848,0.714224,0.141956,0.115433,0.705757,0.75,0.857143,0.5,0.000000,0.000000,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.212746
307508,456253,0,0.0,0.0,0.0,1.0,0.211963,0.602688,0.575672,0.579849,1.000000,1.000000,0.25,0.4,0.2,0.067416,0.578523,0.726937,0.284424,0.509220,1.0,0.0,0.0,1.0,0.555556,0.000000,1.0,1.0,0.666667,0.0,0.0,0.0,1.0,0.684211,0.820989,0.6

Since the IDs of the loanees aren't relevant to whether they will default, we will remove this column. Then, we split our data into a small evaluation set (4 values) and split the remaining values 80/20 into train and test sets.

In [62]:
# identify target and unneeded features
y_1 = app_data_1['TARGET']
ids_1 = app_data_1['SK_ID_CURR']
X_1 = app_data_1.drop(columns=['TARGET','SK_ID_CURR'])

# split into train, test, evalutation sets
X_11, X_eval, y_11, y_eval = train_test_split(X_1, y_1, test_size = .00001, random_state=0)
X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_11, y_11, test_size = 0.2)

# show evaluation default values
pd.pandas.set_option("display.max_columns", None)
print("Target variables of evaluation set: ")
print(y_eval)

Target variables of evaluation set: 
42962     1
227307    0
290035    0
239833    0
Name: TARGET, dtype: int64


Creating a Rondom Forest with code adapted from a DataCamp article. The Random Forest runs a given number of descision trees (here 50) that then "vote" on the predicted outcome.

In [63]:
# /***************************************************************************************
# *    Title: Understanding Random Forests Classifiers in Python
# *    Author: Avinash Navlani
# *    Date: May 16th, 2018
# *    Code version: Unknown
# *    Availability: https://www.datacamp.com/community/tutorials/random-forests-classifier-python
# *
# ***************************************************************************************
forest_1 = RandomForestClassifier(n_estimators = 50, verbose=1) # build random forest
forest_1.fit(X_train_1, y_train_1) # fit forest on training data

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:   49.8s finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=50,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=1, warm_start=False)

Now our forest will predict the values on each set of train, test, and evaluation values, and sklearn will compute their accuracy scores.
- We do this for the test set to see how the forest generalizes to new data.
- The train set's scores are mainly to compare to the test set's scores and check for overfitting.
- As for the evaluation set, since it is a very small subset we'll look directly at those predictions later.

In [65]:
y_trainset_pred_1 = forest_1.predict(X_train_1)
y_pred_1 = forest_1.predict(X_test_1)
y_eval_pred = forest_1.predict(X_eval)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    5.1s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    1.3s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:    0.0s finished


In [67]:
print("Forest Train Accuracy: ", metrics.accuracy_score(y_train_1, y_trainset_pred_1))
print("Forest Test Accuracy: ", metrics.accuracy_score(y_test_1, y_pred_1))
print("Forest Eval Accuracy: ", metrics.accuracy_score(y_eval, y_eval_pred))

Forest Train Accuracy:  0.9993211520091055
Forest Test Accuracy:  0.9200838997105785
Forest Eval Accuracy:  0.75


Now, we'll delve deeper into that Eval score by analysing two loanees with differing default values.

In [71]:
prediction = y_eval_pred[0:3]

In [72]:
print("Forest's prediction of ID#42962: ", prediction[0])
print("Forest's prediction of ID#227307: ", prediction[1])
print("Actual values, respectively: ", y_eval.iloc[0:2].values)

Forest's prediction of ID#42962:  0
Forest's prediction of ID#227307:  0
Actual values, respectively:  [1 0]
